In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import scipy.integrate as intg

### This code follows the Zhang 2018 paper on IA self-calibration

#### - the "chi's" are all computed in unit of c/H0

In [ ]:
#constants
Omega0 = 0.3 #present day matter density in unit of critical density
OmegaLambda = 0.7 
Omegak = 1-Omega0-OmegaLambda

deltaz = 0.2 #redshift bin width
z_limit = 20

#functions
def z_i_center(i): #the center for the ith redshift bin
    return 0.2*(i+1)

In [ ]:
#comoving distance function, in unit of c/H0
#Troxel and Ishak 2014 equation (8) without the factor of H0
def chi_integrand(z):
    #import numpy as np
    return 1/np.sqrt(Omega0*(1+z)**3+Omegak*(1+z)**2+OmegaLambda)
def chi(z):
    #import numpy as np
    #import scipy.integrate as intg
    return intg.quad(chi_integrand,0,z)[0]
z_test = np.linspace(0,z_limit,500)
chi_test = np.array([])
for z in z_test: chi_test = np.append(chi_test,chi(z))
plt.plot(z_test, chi_test)
plt.xlabel('redshift z')
plt.ylabel('comoving distance chi(z) \n ; Omega0=%1.2f, OmegaLambda=%1.2f' %(Omega0,OmegaLambda))
plt.show()

In [4]:
#Gaussian Photo-Z PDF
def sigma_p(z):
    #Zhang 2018 discusses when 0.05->0.03 accuracy improves by 2.3x on page 7 bottom right
    return 0.05*(1+z)
def photozPDF(ztrue,zphoto):
    #??? The photo-z PDF has sigma_p(z), is it sigma_p(ztrue) or sigma_p(zphoto)?
    return 1/(np.sqrt(np.pi)*sigma_p(ztrue))*np.exp(-((zphoto-ztrue)/sigma_p(ztrue))**2)
# #testing the gaussian PDF normalization
# def gaussian_test(x,center=3,width=4):
#     import numpy as np
#     return 1/(np.sqrt(np.pi)*width)*np.exp(-((x-center)/width)**2)
# z_test = np.linspace(-10,10,500)
# n_z_test = gaussian_test(z_test,3,4)
# plt.ylim(0,0.3)
# plt.plot(z_test, n_z_test)
# intg.quad(gaussian_test,-100,100)

In [5]:
#Lensing kernel
def W_L(z_L,z_G):
    if z_L<z_G:
        return 3*Omega0/2 * (1+z_L)*chi(z_L)*(1-chi(z_L)/chi(z_G))
    else: return 0

In [27]:
#The Q function is approximately eta (Appendix B eqn B5) ~ eta function
#Computing eta
    #1. determine a redshift bin of interest, width 0.2
    #2. normalize the n(z) distribution
    #3. calculating upper and lower integral, and divide
i = 5
z_lowerbound = z_i_center(i)-deltaz/2
z_upperbound = z_i_center(i)+deltaz/2
print("calculating redshift bin %1.2f - %1.2f" %(z_lowerbound,z_upperbound))

#redshift distributions
def n_z(z):
    return z*z*np.exp(-z/0.5)
#galaxy distribution normalized in bin i    
n_z_i_norm = intg.quad(n_z,z_lowerbound,z_upperbound)
def n_z_i(z):
    return z*z*np.exp(-z/0.5)/n_z_i_norm
#galaxy distribution normalized from 0 to infinity~z_limit
n_z_all_norm = intg.quad(n_z,0,z_limit)
def n_z_all(z):
    return z*z*np.exp(-z/0.5)/n_z_all_norm
    
z_intg_range = np.linspace(0,z_limit,20)

for z in z_intg_range:
    
def upper_integrand(z_G,zphoto_g,zphoto_G,z): 
    #Need z_L, and z_g from global scope
    #setting z_L ~ z_g = z

    #defined for the ith redshift bin
    #the integration is performed on zphoto_g,z_G,and zphoto_G, after the integration, the dependency is on z_L ~= z_g only.
    #factor of 2 included
    if zphoto_G<zphoto_g:
        #return 2*W_L(z_L,z_G)*photozPDF(z_G,zphoto_G)*photozPDF(z_g,zphoto_g)*n_z_i(zphoto_G)*n_z_i(zphoto_g)
        return 2*W_L(z,z_G)*photozPDF(z_G,zphoto_G)*photozPDF(z,zphoto_g)*n_z_i(zphoto_G)*n_z_i(zphoto_g)
    else: 
        return 0
# def upper_integral():
#     #defined for the ith redshift bin
#     return intg.nquad(upper_integrand,[[0,z_limit],[z_lowerbound,z_upperbound],[z_lowerbound,z_upperbound],[0,z_limit]])

intg.tplquad(upper_integrand,[z_lowerbound,z_upperbound],[z_lowerbound,z_upperbound],[0,z_limit]])
# def upper_integrand(x1,x2,x3,x4):
#     return x4
# intg.nquad(upper_integrand,[[0,1],[0,1],[0,1],[0,4]])

calculating redshift bin 1.10 - 1.30


TypeError: tplquad() missing 5 required positional arguments: 'b', 'gfun', 'hfun', 'qfun', and 'rfun'

In [105]:




def lower_integrand(z_G,zphoto_g,zphoto_G): 
    #need i from global scope
    
    return W_L(z_L,z_G)*photozPDF(z_G,zphoto_G)*photozPDF(z_g,zphoto_g)*n_z(zphoto_G)*n_z(zphoto_g)[0]
def lower_integral(z_L,z_g,i):
    #need i from global scope
    #ith redshift bin
    
    lowerbound = z_i_center(i)-deltaz/2
    upperbound = z_i_center(i)+deltaz/2
    return intg.tplquad(lower_integrand,lowerbound,upperbound,lowerbound,upperbound,0,z_limit)[0]

def eta(z):
    #need i from global scope
    return upper_integral(z,z,i)/lower_integral(z,z,i)
    
    


In [ ]:
"distribution normalization testings"
# #galaxy number density distribution (LSST oriented)
# #normalized from 0 - infinity
# def n_z(z):
#     #import numpy as np
#     return z*z*np.exp(-z/0.5)
# #redshift distribution tests
# z_test = np.linspace(0,z_limit,500)
# n_z_test = n_z(z_test)
# n_z_normalize_const = intg.quad(n_z,0,z_limit)[0]
# print("for z_limit = ",z_limit,", n_z has normalization constant = ", n_z_normalize_const)
# print("for z_limit = ",z_limit,", we renormalize n(z)")
# def n_z(z):
#     #import numpy as np
#     return z*z*np.exp(-z/0.5)/n_z_normalize_const
# plt.plot(z_test, n_z_test)
# plt.xlabel('redshift z')
# plt.ylabel('normalized redshift distribution n(z)')
# plt.show()

In [41]:
i = 5
z_lowerbound = z_i_center(i)-deltaz/2
z_upperbound = z_i_center(i)+deltaz/2
print("calculating redshift bin %1.2f - %1.2f" %(z_lowerbound,z_upperbound))

#redshift distributions
def n_z(z):
    return z*z*np.exp(-z/0.5)
#galaxy distribution normalized in bin i    
n_z_i_norm = intg.quad(n_z,z_lowerbound,z_upperbound)
def n_z_i(z):
    return z*z*np.exp(-z/0.5)/n_z_i_norm
#galaxy distribution normalized from 0 to infinity~z_limit
n_z_all_norm = intg.quad(n_z,0,z_limit)
def n_z_all(z):
    return z*z*np.exp(-z/0.5)/n_z_all_norm

intg_steps = 21
z_intg_range = np.linspace(0,z_limit,intg_steps)
for i in range(intg_steps-1):
    z_intg_midpoint = (i+0.5)*z_limit/(intg_steps-1) #z value between z_intg_range[i] and z_intg_range[i+1]
    print(z_midpoint)
    
    def upper_integrand(z_G,zphoto_g,zphoto_G): #defined for the ith redshift bin
        #setting z_L ~ z_g = z_intg_midpoint
        #the integration is performed on zphoto_g,z_G,and zphoto_G, after the integration, the dependency is on z_L ~= z_g only.
        if zphoto_G<zphoto_g:
            return 2*W_L(z_intg_midpoint,z_G)*photozPDF(z_G,zphoto_G)*photozPDF(z_intg_midpoint,zphoto_g)*n_z_i(zphoto_G)*n_z_i(zphoto_g)
        else: return 0
    def lower_integrand(z_G,zphoto_g,zphoto_G): #defined for the ith redshift bin
        return W_L(z_intg_midpoint,z_G)*photozPDF(z_G,zphoto_G)*photozPDF(z_intg_midpoint,zphoto_g)*n_z_i(zphoto_G)*n_z_i(zphoto_g)
    upper_integral = intg.tplquad(upper_integrand,z_lowerbound,z_upperbound,z_lowerbound,z_upperbound,0,z_limit)
    lower_integral = intg.tplquad(lower_integrand,z_lowerbound,z_upperbound,z_lowerbound,z_upperbound,0,z_limit)
    
    print(upper_integral,lower_integral)
    #eta = upperintegrand evaluated at z_midpoint / lower integrand evaluated at z_midpoint


calculating redshift bin 1.10 - 1.30
19.5


TypeError: only size-1 arrays can be converted to Python scalars